Seting Up the Data with a dataframe that has the tokens column added:

In [43]:
import pandas as pd
from _DevPatentPriorArtFinder import _DevPatentPriorArtFinder as d
from scipy.constants import pt
from unittest import TestCase
import re
import math
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [1]:



myCsv ='https://raw.githubusercontent.com/GabeAspir/Patent-Prior-Art-Finder/main/resourcesZF/TenNewPatents.csv'


# I added '' to the stopwords to avoid the case where a short first word turns into an empty list item
def _tokenize(dataframe):
    dataframe['Tokens'] = dataframe['Abstract'].apply(_tokenizeText)


# Will add column to dataframe called 'Tokens'
# Gabe
def _tokenizeText(string):

    def filterOut(word):
        remove_list = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                       'ourselves', 'you', "you're", "you've", "you'll", "you'd",
                       'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his',
                       'himself', 'she', "she's", 'her', 'hers', 'herself', 'it',
                       "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs',
                       'themselves', 'what', 'which', 'who', 'whom', 'this', 'that',
                       "that'll", 'these', 'those', 'am', 'is', 'are', 'was',
                       'were', 'be', 'been', 'being', 'have', 'has', 'had',
                       'having', 'do', 'does', 'did', 'doing', 'a', 'an',
                       'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
                       'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against',
                       'between', 'into', 'through', 'during', 'before', 'after', 'above',
                       'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over',
                       'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when',
                       'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
                       'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own',
                       'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just',
                       'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o',
                       're', 've', 'y', 'ain', 'aren', "aren't",
                       'couldn', "couldn't", 'didn', "didn't", 'doesn',
                       "doesn't", 'hadn', "hadn't", 'hasn', "hasn't",
                       'haven', "haven't", 'isn', "isn't", 'ma', 'mightn',
                       "mightn't", 'mustn', "mustn't", 'needn', "needn't",
                       'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't",
                       'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

        if word in remove_list:
            return False
        else:
            return True

    lowercasedString = string.lower()
    # To split based on white space and random characters
    stringArray = re.split('\W+', lowercasedString)
    # Will substitute numbers for _NUM_
    stringArray = [re.sub(r"[0-9]+", "_NUM_", s) for s in stringArray]
    # Will filter out 1 letter words like "I" and "a"
    stringArray = list(filter(lambda s: len(s) > 1, stringArray))
    fullyFiltered = list(filter(filterOut, stringArray))
    # Will return a List/Array
    return fullyFiltered

def getCorpus(dataframe):
          corpus = set()
          for r in dataframe['Tokens']:
              corpus.update(r)
          return corpus
    

df = pd.read_csv(myCsv)
_tokenize(df)
corpus = getCorpus(df)
df




  






,Publication_Number,Abstract,Description,Claim,Tokens
0,US-2016057569-A1,Mobile devices can provide app recommendations...,RELATED APPLICATIONS \n This application i...,What is claimed is: \n \n 1 . A me...,"[mobile, devices, provide, app, recommendation..."
1,US-6786308-B1,An anti-lock brake system for a bicycle having...,BACKGROUND OF THE INVENTION \n 1. Field of...,What is claimed is: \n \n 1. An ant...,"[anti, lock, brake, system, bicycle, sliding, ..."
2,US-5913778-A,A basketball mounting system for an in-ground ...,BACKGROUND OF THE INVENTION \n 1. Field of...,What is claimed is: \n \n 1. A mount...,"[basketball, mounting, system, ground, basketb..."
3,US-7115842-B2,Electric blankets and methods and systems for ...,This is a division of U.S. application Ser. No...,"1. An electric blanket, said electric blanket ...","[electric, blankets, methods, systems, making,..."
4,US-7374503-B2,When a rod extending out of the bottom of a ho...,FIELD OF THE INVENTION \n The field of this ...,1. A collapsible basketball backboard assembly...,"[rod, extending, bottom, hollow, rigid, plasti..."
5,US-9073532-B2,A homeostatic flying hovercraft preferably uti...,RELATED APPLICATIONS \n This application i...,The invention claimed is: \n \n 1. A...,"[homeostatic, flying, hovercraft, preferably, ..."
6,US-2016094628-A1,An app store includes a plurality of state mac...,CROSS-REFERENCE TO RELATED APPLICATIONS \n ...,What is claimed is: \n \n 1 . A sy...,"[app, store, includes, plurality, state, machi..."
7,US-2015281222-A1,A Time-based One-Time Password (TOTP) validato...,BACKGROUND \n Many companies are using mul...,"1 . A method, comprising:\n receiving, by a Ti...","[time, based, one, time, password, totp, valid..."
8,US-7934412-B2,The present invention relates to the monitorin...,CROSS REFERENCE TO RELATED APPLICATIONS \n ...,1. A method for determining a source of an emi...,"[present, invention, relates, monitoring, gas,..."
9,US-2012029990-A1,A secure element can be physically coupled to ...,RELATED APPLICATIONS \n This application i...,1 . A method for conducting induction triggere...,"[secure, element, physically, coupled, mobile,..."


Now Testing the bag of words method we are using 

In [9]:
dataframe2 = df.copy(deep=True)

def _bagOfWordize(dataframe, corpus):
  counts = []
  for row in dataframe['Tokens']:
    count = []  # Initialize count as an empty list
    for word in corpus:
      count.append(row.count(word))  # get the wordcount in each list of words, and record the count
    counts.append(count)  # Each list of wordCount vectors represents one document, and the counts variable is the list of all our docs' counts
  dataframe['BagOfWords'] = counts


_bagOfWordize(dataframe2, corpus)
dataframe2


,Publication_Number,Abstract,Description,Claim,Tokens,BagOfWords
0,US-2016057569-A1,Mobile devices can provide app recommendations...,RELATED APPLICATIONS \n This application i...,What is claimed is: \n \n 1 . A me...,"[mobile, devices, provide, app, recommendation...",
1,US-6786308-B1,An anti-lock brake system for a bicycle having...,BACKGROUND OF THE INVENTION \n 1. Field of...,What is claimed is: \n \n 1. An ant...,"[anti, lock, brake, system, bicycle, sliding, ...",
2,US-5913778-A,A basketball mounting system for an in-ground ...,BACKGROUND OF THE INVENTION \n 1. Field of...,What is claimed is: \n \n 1. A mount...,"[basketball, mounting, system, ground, basketb...",
3,US-7115842-B2,Electric blankets and methods and systems for ...,This is a division of U.S. application Ser. No...,"1. An electric blanket, said electric blanket ...","[electric, blankets, methods, systems, making,...",
4,US-7374503-B2,When a rod extending out of the bottom of a ho...,FIELD OF THE INVENTION \n The field of this ...,1. A collapsible basketball backboard assembly...,"[rod, extending, bottom, hollow, rigid, plasti...",
5,US-9073532-B2,A homeostatic flying hovercraft preferably uti...,RELATED APPLICATIONS \n This application i...,The invention claimed is: \n \n 1. A...,"[homeostatic, flying, hovercraft, preferably, ...",
6,US-2016094628-A1,An app store includes a plurality of state mac...,CROSS-REFERENCE TO RELATED APPLICATIONS \n ...,What is claimed is: \n \n 1 . A sy...,"[app, store, includes, plurality, state, machi...",
7,US-2015281222-A1,A Time-based One-Time Password (TOTP) validato...,BACKGROUND \n Many companies are using mul...,"1 . A method, comprising:\n receiving, by a Ti...","[time, based, one, time, password, totp, valid...",
8,US-7934412-B2,The present invention relates to the monitorin...,CROSS REFERENCE TO RELATED APPLICATIONS \n ...,1. A method for determining a source of an emi...,"[present, invention, relates, monitoring, gas,...",
9,US-2012029990-A1,A secure element can be physically coupled to ...,RELATED APPLICATIONS \n This application i...,1 . A method for conducting induction triggere...,"[secure, element, physically, coupled, mobile,...",


Using my personal implementation to test against it

In [14]:
def bow(data,vocab):
	
  if 'BagOfWords' not in data.columns:
	  data.insert(len(data.columns), 'BagOfWords', '')

  for i in data.index:
	  tokens = data['Tokens'][i]
	  vector = vectorize(tokens,vocab)
	  data['BagOfWords'][i] =vector

  return data


def vectorize(tokens,vocab):
	vector = []
	for word in vocab:
		vector.append(tokens.count(word))

	return vector

bow(df, corpus)
df


,Publication_Number,Abstract,Description,Claim,Tokens,BagOfWords
0,US-2016057569-A1,Mobile devices can provide app recommendations...,RELATED APPLICATIONS \n This application i...,What is claimed is: \n \n 1 . A me...,"[mobile, devices, provide, app, recommendation...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,US-6786308-B1,An anti-lock brake system for a bicycle having...,BACKGROUND OF THE INVENTION \n 1. Field of...,What is claimed is: \n \n 1. An ant...,"[anti, lock, brake, system, bicycle, sliding, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,US-5913778-A,A basketball mounting system for an in-ground ...,BACKGROUND OF THE INVENTION \n 1. Field of...,What is claimed is: \n \n 1. A mount...,"[basketball, mounting, system, ground, basketb...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, ..."
3,US-7115842-B2,Electric blankets and methods and systems for ...,This is a division of U.S. application Ser. No...,"1. An electric blanket, said electric blanket ...","[electric, blankets, methods, systems, making,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,US-7374503-B2,When a rod extending out of the bottom of a ho...,FIELD OF THE INVENTION \n The field of this ...,1. A collapsible basketball backboard assembly...,"[rod, extending, bottom, hollow, rigid, plasti...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 5, ..."
5,US-9073532-B2,A homeostatic flying hovercraft preferably uti...,RELATED APPLICATIONS \n This application i...,The invention claimed is: \n \n 1. A...,"[homeostatic, flying, hovercraft, preferably, ...","[0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
6,US-2016094628-A1,An app store includes a plurality of state mac...,CROSS-REFERENCE TO RELATED APPLICATIONS \n ...,What is claimed is: \n \n 1 . A sy...,"[app, store, includes, plurality, state, machi...","[0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, ..."
7,US-2015281222-A1,A Time-based One-Time Password (TOTP) validato...,BACKGROUND \n Many companies are using mul...,"1 . A method, comprising:\n receiving, by a Ti...","[time, based, one, time, password, totp, valid...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,US-7934412-B2,The present invention relates to the monitorin...,CROSS REFERENCE TO RELATED APPLICATIONS \n ...,1. A method for determining a source of an emi...,"[present, invention, relates, monitoring, gas,...","[1, 1, 2, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, ..."
9,US-2012029990-A1,A secure element can be physically coupled to ...,RELATED APPLICATIONS \n This application i...,1 . A method for conducting induction triggere...,"[secure, element, physically, coupled, mobile,...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, ..."


Checking of they have equal values by making sure the sum of the bag of words for each abstract is equal

In [27]:

def check_for_equal_sum():
  bool = True
  for bow1, bow2 in zip(df['BagOfWords'], dataframe2['BagOfWords']):
    if sum(bow1)!= sum(bow2):
      bool = False

  if bool: 
    print("Test Passed")

  else:
    print("Test Failed")


check_for_equal_sum()


Test Passed


Now Testing Cosine Similarity: Our Implementation:

In [33]:
def cosineTable(dataframe):
        """
         Takes a dataframe and returns a table with the cosine simialrity metrics of each pair
        , uses scikit-learn's cosine function.

        :param dataframe: dataframe from init
        :return: pandas table with cosine similarity (SciKit implementation) index from BOW vectors
        """

        newTable = pd.DataFrame(cosine_similarity(dataframe['BagOfWords'].tolist()))
        newTable.columns = dataframe['Publication_Number']
        newTable.index = dataframe['Publication_Number']

        return newTable


cs_table_1 = cosineTable(df)
cs_table_1

Publication_Number,US-2016057569-A1,US-6786308-B1,US-5913778-A,US-7115842-B2,US-7374503-B2,US-9073532-B2,US-2016094628-A1,US-2015281222-A1,US-7934412-B2,US-2012029990-A1
Publication_Number,,,,,,,,,,
US-2016057569-A1,1.000000,0.000000,0.000000,0.019929,0.000000,0.000000,0.176833,0.190792,0.000000,0.193966
US-6786308-B1,0.000000,1.000000,0.056699,0.061446,0.048256,0.028330,0.050522,0.000000,0.000000,0.007323
US-5913778-A,0.000000,0.056699,1.000000,0.015678,0.055406,0.075896,0.039477,0.007262,0.000000,0.019618
US-7115842-B2,0.019929,0.061446,0.015678,1.000000,0.030499,0.000000,0.055879,0.000000,0.000000,0.032397
US-7374503-B2,0.000000,0.048256,0.055406,0.030499,1.000000,0.024608,0.021942,0.021192,0.000000,0.012722
US-9073532-B2,0.000000,0.028330,0.075896,0.000000,0.024608,1.000000,0.075141,0.009676,0.000000,0.008713
US-2016094628-A1,0.176833,0.050522,0.039477,0.055879,0.021942,0.075141,1.000000,0.135895,0.034756,0.064097
US-2015281222-A1,0.190792,0.000000,0.007262,0.000000,0.021192,0.009676,0.135895,1.000000,0.000000,0.105053
US-7934412-B2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034756,0.000000,1.000000,0.024181


My implementation original used to test against it: 

In [41]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity as cs

def cosine_similarity_func(vectorA, vectorB):
  v1 = np.array(vectorA).reshape(1,-1)
  v2 = np.array(vectorB).reshape(1,-1)
  return cs(v1, v2)[0][0]

#Parameter is a dataframe
def table_of_cos(data):
  table_of_cosines = pd.DataFrame(df['Publication_Number'])
  for bow, n in zip(data['BagOfWords'], data['Publication_Number']):
    number = n #getting the publication number so can use it as header later on 
    comps = [] #series that represents this bag of word's cosine comp with all bow's
    for b in data['BagOfWords']:
      comps.append(cosine_similarity_func(bow,b)) 
    table_of_cosines[n]= comps

  return table_of_cosines


cs_table_2 = table_of_cos(df)
cs_table_2

,Publication_Number,US-2016057569-A1,US-6786308-B1,US-5913778-A,US-7115842-B2,US-7374503-B2,US-9073532-B2,US-2016094628-A1,US-2015281222-A1,US-7934412-B2,US-2012029990-A1
0,US-2016057569-A1,1.000000,0.000000,0.000000,0.019929,0.000000,0.000000,0.176833,0.190792,0.000000,0.193966
1,US-6786308-B1,0.000000,1.000000,0.056699,0.061446,0.048256,0.028330,0.050522,0.000000,0.000000,0.007323
2,US-5913778-A,0.000000,0.056699,1.000000,0.015678,0.055406,0.075896,0.039477,0.007262,0.000000,0.019618
3,US-7115842-B2,0.019929,0.061446,0.015678,1.000000,0.030499,0.000000,0.055879,0.000000,0.000000,0.032397
4,US-7374503-B2,0.000000,0.048256,0.055406,0.030499,1.000000,0.024608,0.021942,0.021192,0.000000,0.012722
5,US-9073532-B2,0.000000,0.028330,0.075896,0.000000,0.024608,1.000000,0.075141,0.009676,0.000000,0.008713
6,US-2016094628-A1,0.176833,0.050522,0.039477,0.055879,0.021942,0.075141,1.000000,0.135895,0.034756,0.064097
7,US-2015281222-A1,0.190792,0.000000,0.007262,0.000000,0.021192,0.009676,0.135895,1.000000,0.000000,0.105053
8,US-7934412-B2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034756,0.000000,1.000000,0.024181
9,US-2012029990-A1,0.193966,0.007323,0.019618,0.032397,0.012722,0.008713,0.064097,0.105053,0.024181,1.000000
